In [1]:
import pandas as pd
import numpy as np
from comparison_plot import comparison_plot

In [2]:
nr_models = 10
nr_intercepts = 5

intercept_tuples = [str(i) for i in range(nr_intercepts)]
short_controls = ['age', 'grade', 'iq_score', 'gender']
long_controls = short_controls + [
    'parent_educ', 'parent_inc', 'parent_occup', 'dist_to_school', 
    'single_parent', 'single_parent_x_parent_inc']

cols = ['model', 'param_value', 'param_name', 'lower', 'upper', 'group']

model_comp_df = pd.DataFrame()

for i in range(nr_models):
    if i < 0.7 * nr_models:
        ctrl_names = short_controls
        factor = 0.5
    else:
        ctrl_names = long_controls
        factor = 0.7
    nr_params = nr_intercepts + len(ctrl_names)
    
    df = pd.DataFrame(columns=cols)
    df['param_value'] = factor * np.arange(nr_params) + np.random.normal(0, 1, nr_params)
    df['param_name'] = [str(i) for i in range(nr_intercepts)] + ctrl_names
    df['lower'] = df['param_value'] - np.random.normal(1, factor, nr_params)
    df['upper'] = df['param_value'] + np.random.normal(1, factor, nr_params)
    df['group'] = ['intercept'] * nr_intercepts + [r'$\beta$'] * len(ctrl_names)
    df['model'] = 'model_' + str(i)
    
    model_comp_df = pd.concat([model_comp_df, df], axis=0, sort=False)
    
model_comp_df.reset_index(inplace=True, drop=True)
model_comp_df = model_comp_df[['model', 'group', 'param_name', 'lower', 'param_value', 'upper']]
model_comp_df[:9]

,model,group,param_name,lower,param_value,upper
0,model_0,intercept,0,-0.468476,0.302937,1.615633
1,model_0,intercept,1,-1.871790,-0.995386,-0.029428
2,model_0,intercept,2,1.537369,1.700074,3.005390
3,model_0,intercept,3,2.071727,2.745706,4.434936
4,model_0,intercept,4,1.745859,2.511780,3.868747
5,model_0,$\beta$,age,3.266969,3.737981,4.288490
6,model_0,$\beta$,grade,2.605270,3.076004,3.941476
7,model_0,$\beta$,iq_score,1.816652,2.997151,4.680116
8,model_0,$\beta$,gender,4.023643,4.804909,5.911233


In [4]:
comparison_plot(df=model_comp_df)

Loading BokehJS ...